In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
import operator
#sys.setrecursionlimit(20000)

In [2]:
#FILE_NAME = "test.txt"
#FILE_NAME = "test2.txt"
FILE_NAME = "input.txt"
DIAG=False
DIAG2 = False
ARROWS = False

In [3]:
CHAR_MAP = {".":1, "#":0,">":2,"<":3,"^":4,"v":5}
ARROW_DIRECT = {2:(0,1),3:(0,-1),4:(-1,0),5:(1,0)}

def getCharFromNum(num):
    for x,y in CHAR_MAP.items():
        if y == num:
            return x

def getNumFromChar(sym):
    return CHAR_MAP[sym]

def getDirectionNoArrows(symNum):
    return None
    
    
def getDirectionWithArrows(symNum):
    if symNum in ARROW_DIRECT:
        return ARROW_DIRECT[symNum]
    else:
        return None

def getDirection(symNum):
    if ARROWS:
        return getDirectionWithArrows(symNum)
    else:
        return getDirectionNoArrows(symNum)

In [4]:
def buildArrayFromLines(lines):
    
    tmp_arr = []
    for line in lines:
        tmp_line = []    
        
        for c in line:
            if c != "\n":
                tmp_line += [CHAR_MAP[c]]
        tmp_arr.append(tmp_line)
    
    
    return np.array(tmp_arr)
    

In [5]:
c=np.array([[1,2,3],[2,4,2]])
id1,id2=np.where(c==2)

for i in range(0,len(id1)):
    print(id1[i],id2[i],c[id1[i],id2[i]])

z=list(zip(id1,id2))
z

0 1 2
1 0 2
1 2 2


[(0, 1), (1, 0), (1, 2)]

In [6]:
def getNeighbors(x,y,my_arr,my_nodes):
    DIRS = [(1,0),(-1,0),(0,-1),(0,1)]
    res = []
    shX,shY = my_arr.shape
    

    
    for d in DIRS:
        x_new = x + d[0]
        y_new = y + d[1]
        if (x_new,y_new) in my_nodes: 
            
            el = my_arr[x,y]
            dir_sym = getDirection(el)
            
            if dir_sym is None or dir_sym == d:
                res += [(x_new,y_new)]
            else:
                # arrow doesn't allow to move in this direction
                continue
        
    return res

In [7]:
def buildGraphFromArray(my_arr):
    shX,shY=my_arr.shape
    
    idx,idy = np.where(my_arr != CHAR_MAP["#"])
    my_nodes = list(zip(idx,idy))
    if DIAG: print("Nodes: ",my_nodes)
    
    my_edges = []
    for x in range(0,shX):
        for y in range(0,shY):
            if (x,y) in my_nodes:
                neigh = getNeighbors(x,y,my_arr,my_nodes)
                my_edges += [((x,y),neigh_point,1) for neigh_point in neigh]
  
    return my_edges

In [8]:
def getNodes(my_edges):
    my_nodes = set([x for x,y,_ in my_edges]+[y for x,y,_ in my_edges])
    return list(my_nodes)
   

In [9]:
def getStartAndEndPoints(myArr):
    shX,_=myArr.shape
    st_Y = np.nonzero(myArr[0]==getNumFromChar("."))
    end_Y = np.nonzero(myArr[shX-1]==getNumFromChar("."))
    
    return ((0,int(st_Y[0])),(shX-1,int(end_Y[0])))

In [10]:
def collapseEdges(my_edges,st_point,end_point):
    nodes = getNodes(my_edges)
    for n in nodes:
        if n == st_point or n==end_point:
            # cannot collapse source or target
            continue
        else:
            # if has max 2 outgoing edges and 2 incoming edges and these to the same neighbors
            neighb_in = set([(x,z) for (x,y,z) in my_edges if y == n])
            neighb_out = set([(y,z) for (x,y,z) in my_edges if x == n])
            neighb_all=neighb_in.copy()
            neighb_all |= neighb_out
            
            canCollapse = True
            if len(neighb_in) < 1 or len(neighb_in) >2: canCollapse=False
            if len(neighb_out) < 1 or len(neighb_out) >2: canCollapse=False
            if len(neighb_all) > 2: canCollapse = False
                
            if canCollapse:
                for (x,zx) in neighb_in:
                    for (y,zy) in neighb_out:
                        if x != y:
                            # remove (x,n,zx) and (y,n,zy) from edges
                            # replace with (x,y,zx+zy)
                            my_edges.remove((x,n,zx))
                            my_edges.remove((n,y,zy))
                            my_edges.append((x,y,zx+zy))
            
    return my_edges

In [11]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    my_arr = buildArrayFromLines(lines)
    st_point,end_point = getStartAndEndPoints(my_arr)
    
    my_edges = buildGraphFromArray(my_arr)
    my_edges = collapseEdges(my_edges,st_point,end_point)
    

    
    return my_arr,my_edges,st_point,end_point


# Get map as a graph and collapse 

In [12]:
import sys


TARGET = 1000000000

np.set_printoptions(threshold=sys.maxsize)

my_map,my_edges,st_point,end_point = parseFile(FILE_NAME)


#my_nodes = getNodes(my_edges)

   

In [13]:
a = [ x for x,_,_ in my_edges]
b= [ y for _,y,_ in my_edges]
node_count = len(set(a + b))
edge_count = len(my_edges)

print("Graph has {} nodes and {} edges.".format(node_count,edge_count))

Graph has 36 nodes and 120 edges.


# Recursion

In [15]:
def getNeighborsFromEdges(p_cur,my_edges):
    return [(p_tgt,edge_len) for (x,p_tgt,edge_len) in my_edges if x == p_cur]

def findNaiveRecur(p_cur,p_en,seen_nodes, len_sofar,my_edges):
    seen_nodes_copy = seen_nodes.copy()
    if p_cur in seen_nodes_copy:
        # this path loops on itself, not good
        return -1
    elif p_cur == p_en:
        # end of path
        return len_sofar
    else:
        if DIAG: print("Checking point ",p_cur)
        seen_nodes_copy += [p_cur]
        res = -1 # if there are no neighbours...
        if DIAG: print("  Neighbor count",len(dict_edges[p_cur]))
        for neighb_node,edge_len in getNeighborsFromEdges(p_cur,my_edges):
            if neighb_node not in seen_nodes_copy:
                res_tmp = findNaiveRecur(neighb_node,p_en,seen_nodes_copy,len_sofar + edge_len,my_edges)
                
                if res_tmp > res: res = res_tmp
                
        return res
    return -1

def findNaiveOuterRecur(p_st,p_en,my_edges):
    return findNaiveRecur(p_st,p_en,[],0,my_edges)

res = findNaiveOuterRecur(st_point,end_point,my_edges)
print("Longest path via recursion:",format(res))

Longest path via recursion: 6490


# Stack

In [18]:
def getUnseenNeighborsFromEdges(p_cur,my_edges,seen):
    neighb_with_len = getNeighborsFromEdges(p_cur,my_edges)    
    seen_nodes =  list(map(operator.itemgetter(0), seen))
    unseen_neighb = [(x,ll) for x,ll in neighb_with_len if x not in seen_nodes]
    if DIAG: print(">>",neighb_with_len)
    if DIAG: print(">>>",seen_nodes)
    if DIAG: print(">>>>",unseen_neighb)
    return unseen_neighb


def findNaiveStack(p_st,p_en,my_edges):
    # initial state
    MAX_LEN = -1    
    PATH_LENGTHS = []
    SEEN = []
    STACK = [(p_st,0,0)]
        
    # loop
    while STACK:
        # pop element off the stack
        p_cur,edge_len,seen_num = STACK[-1]
        STACK = STACK[:-1]
    
        # cut down the path of seen nodes, and get its current (edge) length/distance        
        SEEN = SEEN[:seen_num]
        if (SEEN):
            _,prev_len = SEEN[-1]
        else:
            prev_len = 0
        new_len = prev_len+edge_len
        SEEN+=[(p_cur,new_len)]
        
        # will be checking if something has been seen on popping in !!!
        if p_cur == p_en:
            if new_len>MAX_LEN:
                MAX_LEN = new_len
            PATH_LENGTHS+= [new_len]
        #elif p_cur not in dict_edges:
        #    print ("This shouldn't have happened!!! (point {})".format(p_cur))
        else:            
            
            for neighb_node,edge_len in getUnseenNeighborsFromEdges(p_cur,my_edges,SEEN):                
                STACK += [(neighb_node,edge_len,seen_num+1)]
       
    return MAX_LEN,PATH_LENGTHS


In [19]:
max_path,path_lenghts = findNaiveStack(st_point,end_point,my_edges)
print("Longest path via stack:",max_path)

Longest path via stack: 6490
